In [1]:
import pandas as pd
import numpy as np

layers = [8, 5, 3]
weights = []
bias = []
colNames = ['class', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'nuv_mag']
dataSet = pd.read_csv('GALEX_data-extended-feats.csv', usecols = colNames)

In [2]:
def sig(z):
    return (1)/(1 + np.exp(-z))

def sig_prime(z):
    return sig(z)*(1-sig(z))

def initialiser(layers, weights, bias):
    # initialise all weights and biases throughout the network to small random values
    nLayers = len(layers)
    for num in range(1, nLayers):
        for num2 in range(layers[num]):
            layWeights = np.random.uniform(-0.05, 0.05, layers[num - 1])
            weights.append(layWeights)
    bias.append(np.random.uniform(-0.05, 0.05, sum(layers[1:])))
    return weights, bias



initialiser(layers, weights, bias)

([array([-0.01891674,  0.01576799, -0.01299996, -0.01452863, -0.01859359,
          0.02128298, -0.01357855, -0.04313605]),
  array([ 0.02706956,  0.03355041,  0.048123  ,  0.03301729,  0.00670263,
         -0.0414698 , -0.02344813, -0.03661948]),
  array([-0.04188266,  0.04932861,  0.02615711,  0.03483297, -0.03907388,
          0.01441777,  0.01286104, -0.02486606]),
  array([ 0.0157468 ,  0.01097962,  0.02531218, -0.00539298,  0.02938773,
          0.04872916,  0.01742098, -0.00786166]),
  array([ 0.01540625, -0.00892678,  0.04761241, -0.04382494, -0.04412623,
         -0.04739499,  0.01917141,  0.02831043]),
  array([-0.02532983, -0.04701216,  0.01715933,  0.01052588,  0.00620816]),
  array([-0.00646108, -0.04397024, -0.03290629, -0.00011236,  0.00315546]),
  array([ 0.04653146,  0.04859959, -0.01723641, -0.02859964,  0.03482677])],
 [array([ 0.02154509,  0.01813313,  0.0013882 , -0.00842511,  0.00287031,
         -0.00021216, -0.01360444,  0.01243512])])

In [3]:
def feedFor(x, i, w, b):
    # l should be a two element array, input vector and output vector, ex: [8, 5] or [5, 3]
    base = sum(layers[1:i])
    ele = layers[i]
    newOut = []
    for num in range(ele):
        sigVal = sig(np.dot(w[base + num], x) + b[0][base + num])
        newOut.append(sigVal)
    return newOut

def fixWeights(outputk, outputh, i, w):
    if(dataSet.iloc[i, 0] == 0):
        exp = [1, 0, 0]
    elif(dataSet.iloc[i, 0] == 1):
        exp = [0, 1, 0]
    else:
        exp = [0, 0, 1]
    delK = []; delH = []
    for num in range(layers[-1]):
        delK.append(outputk[num]*(1 - outputk[num])*(exp[num] - outputk[num]))
    for num1 in range(layers[-2]):
        tSum = 0
        for num2 in range(layers[-1]):
            tSum += weights[num1][num2] * delK[num2] 
        delH.append(tSum)
    return delH, delK
        
def all(eta):
    for num in range(7000):
        currRow = dataSet.iloc[num, 1:]
        outputh = feedFor(currRow, 1, weights, bias)
        outputk = feedFor(outputh, 2, weights, bias)
        delH, delK = fixWeights(outputk, outputh, num, weights)
        for num1 in range(5):
            for num2 in range(8):
                weights[num1][num2] += (eta)*(delH[num1])*(currRow[num2])
        for num1 in range(5, 8):
            for num2 in range(5):
                weights[num1][num2] += (eta)*(delK[num1 - 5])*(outputh[num2])
      
all(0.1)
print(*weights, *bias)


c:\users\gaurang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


[-2.45010232e+17 -3.84873955e+16 -2.30764247e+16 -2.45380925e+16
 -2.79558150e+16 -3.12308175e+16 -3.37128756e+16 -2.47695555e+16] [-4.93893014e+16 -7.75831100e+15 -4.65175877e+15 -4.94640259e+15
 -5.63534903e+15 -6.29552590e+15 -6.79586057e+15 -4.99306103e+15] [-5.64047944e+17 -8.86033867e+16 -5.31251687e+16 -5.64901331e+16
 -6.43582101e+16 -7.18977256e+16 -7.76117717e+16 -5.70229934e+16] [3.52268846e+16 5.53360989e+15 3.31786368e+15 3.52801818e+15
 4.01940875e+15 4.49027944e+15 4.84714279e+15 3.56129728e+15] [2.31581615e+17 3.63779632e+16 2.18116429e+16 2.31931991e+16
 2.64236017e+16 2.95191066e+16 3.18651270e+16 2.34119759e+16] [-0.79084348 -0.79515615 -0.74841329 -1.08843607 -1.08872054] [0.45277751 0.39972274 0.42638846 0.4887182  0.49036694] [-0.50778669 -0.50067203 -0.57149448 -0.77539237 -0.70642936] [ 0.02154509  0.01813313  0.0013882  -0.00842511  0.00287031 -0.00021216
 -0.01360444  0.01243512]


In [4]:

np.save('weights', weights)
np.save('bias', bias)